# Installs

## Levenshtein

This may take a while

In [ ]:
!nvidia-smi

Sat Nov 19 20:29:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    45W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install python-Levenshtein
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

!pip install torchsummaryX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 6.4 MB/s 
     |████████████████████████████████| 2.2 MB 69.4 MB/s 
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 2.56 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/thi

## imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"ramanathan16","key":"d4035f55a52974a7c4d6bae8054bc215"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 4.5 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73276 sha256=912c531eb1a885b05374648f642c57945593000043b07e85e62cb41820ffa5af
  Stored in directory: /root/.cache/pip/wheels/de/f7/d8/c3902cacb7e62cb611b1ad343d7cc07f42f7eb76ae3a52f3d1
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!kaggle competitions download -c 11-785-f22-hw3p2

100% 8.88G/8.88G [00:32<00:00, 306MB/s]
100% 8.88G/8.88G [00:32<00:00, 291MB/s]


In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f22-hw3p2.zip  ctcdecode  hw3p2
'''
!unzip -q 11-785-f22-hw3p2.zip
!ls

11-785-f22-hw3p2.zip  ctcdecode  hw3p2	sample_data


In [ ]:
!ls "/content/drive/MyDrive"

ls: cannot access '/content/drive/MyDrive': No such file or directory


# Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE
# This overwrites the phonetics.py file.

CMUdict_ARPAbet = {
    "" : " ", # BLANK TOKEN
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict
mapping = CMUdict_ARPAbet
LABELS = ARPAbet

### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self,data_path,partition= "train"): 
        

        self.data_path = data_path

        if partition == "train":
          self.mfcc_dir = self.data_path/ Path(partition+'-clean-100') /Path('mfcc') 
          self.transcript_dir = self.data_path/ Path(partition+'-clean-100') /Path('transcript')/Path("raw") 

        elif partition == "dev":
          self.mfcc_dir = self.data_path/ Path(partition +'-clean')/Path('mfcc')
          self.transcript_dir = self.data_path/ Path(partition+'-clean')/Path('transcript')/Path("raw")

        self.mfcc_files = (os.listdir(self.mfcc_dir)) # TODO: List files in sefl.mfcc_dir_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load
        self.transcript_files = (os.listdir(self.transcript_dir))

        # self.mfccs, self.transcripts = [], []
        
        # for i in range(0, len(mfcc_files)):
        
        #     mfcc = np.load(self.mfcc_dir/mfcc_files[i])
    
        #     mean = np.mean(mfcc, axis=0)
        #     var = np.std(mfcc,axis=0)
        #     n_mfcc = (mfcc - mean)/var

        #     transcript = np.load(self.transcript_dir/transcript_files[i])[1:-1]
        #     transcript=torch.LongTensor([self.PHONEMES.index(i) for i in transcript])

        #     self.mfccs.append(n_mfcc)
        #     self.transcripts.append(transcript)

        #self.mfccs = np.concatenate(self.mfccs,axis =0)

        #self.transcripts = np.concatenate(self.transcripts,axis =0)

        self.PHONEMES = PHONEMES

        # t = []
        # for tr in self.transcripts:
        #   t.append(np.array([self.PHONEMES.index(i) for i in tr]))
        # self.transcripts = t

        self.length = len(self.transcript_files)



    def __len__(self):
        
        return self.length

    def __getitem__(self, ind):
        

        X = torch.FloatTensor(np.load(os.path.join(self.mfcc_dir, self.mfcc_files[ind])))
        X = (X - X.mean(axis=0))/X.std(axis=0) # cepstral normalization

        Y = np.load(os.path.join(self.transcript_dir, self.transcript_files[ind]))
        Yy = torch.LongTensor([self.PHONEMES.index(i) for i in Y[1:-1]])

        return X, Yy


    def collate_fn(self,batch):
      

        batch_x = [x for x,y in batch]
        batch_y = [y for x,y in batch]

        batch_x_pad = torch.nn.utils.rnn.pad_sequence(batch_x, batch_first=True)
        lengths_x = [x.shape[0] for x in batch_x]

        batch_y_pad = torch.nn.utils.rnn.pad_sequence(batch_y, batch_first=True)
        lengths_y = [y.shape[0] for y in batch_y]

        return batch_x_pad, batch_y_pad, torch.tensor(lengths_x), torch.tensor(lengths_y)

       

### Test Data

In [ ]:
class AudioDatasetTest(torch.utils.data.Dataset):

  def __init__(self,data_path,partition= "test"): 
      
        self.data_path = data_path

        
        self.mfcc_dir = self.data_path/ Path(partition +'-clean')/Path('mfcc')
          
        mfcc_files = sorted(os.listdir(self.mfcc_dir)) 

        self.mfccs = []
        
        for i in range(0, len(mfcc_files)):
        
            mfcc = np.load(self.mfcc_dir/mfcc_files[i])
    
            mean = np.mean(mfcc, axis=0)
            var = np.var(mfcc,axis=0)
            n_mfcc = (mfcc - mean)/var


            self.mfccs.append(n_mfcc)

        #self.mfccs = np.concatenate(self.mfccs,axis =0)

        self.length = len(self.mfccs)
        
  def __len__(self):
        
        return self.length

  def __getitem__(self, ind):
        
        mfcc = self.mfccs[ind] 
        
        return mfcc


  def collate_fn(self,batch):
      
        batch_mfcc = [torch.FloatTensor(x[0]) for x in batch]
        
        batch_mfcc_pad = torch.nn.utils.rnn.pad_sequence(batch_mfcc, batch_first=True, padding_value=0.0)
        
        lengths_mfcc = [len(x) for x in batch_mfcc_pad]

        return batch_mfcc_pad, torch.tensor(lengths_mfcc)

### Data - Hyperparameters

In [ ]:
BATCH_SIZE = 64 # Increase if your device can handle it

transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation

root = '/content/hw3p2' 

### Data loaders

In [ ]:
# get me RAMMM!!!! 
import gc 
gc.collect()

153

In [ ]:
# Create objects for the dataset class
test_data= AudioDatasetTest(root)
train_data = AudioDataset(data_path= root, partition= "train") #TODO
val_data = AudioDataset(data_path=root, partition= "dev") # TODO : You can either use the same class with some modifications or make a new one :)



train_loader = torch.utils.data.DataLoader(train_data, num_workers= 4,
                                           batch_size= BATCH_SIZE,
                                           shuffle= True,collate_fn = train_data.collate_fn)
val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                           batch_size=BATCH_SIZE,
                                           shuffle= False,collate_fn = val_data.collate_fn)
test_loader = torch.utils.data.DataLoader(test_data,
                                           batch_size=BATCH_SIZE,
                                           shuffle= False,collate_fn = test_data.collate_fn)

print("Batch size: ", BATCH_SIZE)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 104014, batches = 1626
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 

torch.Size([64, 1604, 15]) torch.Size([64, 199]) torch.Size([64]) torch.Size([64])


# Model Config

In [ ]:
OUT_SIZE = len(LABELS)
OUT_SIZE

43

## Basic

In [ ]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self):

        super(Network, self).__init__()

        # Adding some sort of embedding layer or feature extractor might help performance.

        self.embedding = nn.Sequential(
              nn.Conv1d(15, 256, 1),
              nn.BatchNorm1d(256),
              nn.GELU(),

              nn.Conv1d(256, 512, 1),
              nn.BatchNorm1d(512),
              nn.GELU(),

              nn.Conv1d(512, 512, 1),
              nn.BatchNorm1d(512),
              nn.GELU(),
        )

        
        # TODO : look up the documentation. You might need to pass some additional parameters.
        self.lstms = nn.Sequential(
            nn.LSTM(input_size = 512, hidden_size = 256, num_layers = 4, bidirectional=True, dropout=0.2, batch_first=True),
        )
       
        # self.lstm = nn.LSTM(input_size = 128, hidden_size = 256, num_layers = 4, batch_first=True)

        self.classification = nn.Sequential(
            nn.Linear(512, 256),
            nn.GELU(),
            nn.Linear(256, 256),
            nn.GELU(),
            nn.Linear(256, OUT_SIZE)
            #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
        )
        
        self.logSoftmax = nn.LogSoftmax(dim=2)#TODO: Apply a log softmax here. Which dimension would apply it on ?

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        # Refer to the handout for hints
        x = torch.permute(x, (0,2,1))
        x= self.embedding(x)
        x = x.permute((0,2,1))
        packed = torch.nn.utils.rnn.pack_padded_sequence(x, lengths=lx, batch_first=True, enforce_sorted=False)
        out,_ = self.lstms(packed)
        unpacked,len_unpacked = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        out = self.classification(unpacked)
        out = self.logSoftmax(out)
        return out, len_unpacked

## INIT

In [ ]:
torch.cuda.empty_cache()

model = Network().to(device)
summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

                              Kernel Shape     Output Shape    Params  \
Layer                                                                   
0_embedding.Conv1d_0          [15, 256, 1]  [64, 256, 1604]    4.096k   
1_embedding.BatchNorm1d_1            [256]  [64, 256, 1604]     512.0   
2_embedding.GELU_2                       -  [64, 256, 1604]         -   
3_embedding.Conv1d_3         [256, 512, 1]  [64, 512, 1604]  131.584k   
4_embedding.BatchNorm1d_4            [512]  [64, 512, 1604]    1.024k   
5_embedding.GELU_5                       -  [64, 512, 1604]         -   
6_embedding.Conv1d_6         [512, 512, 1]  [64, 512, 1604]  262.656k   
7_embedding.BatchNorm1d_7            [512]  [64, 512, 1604]    1.024k   
8_embedding.GELU_8                       -  [64, 512, 1604]         -   
9_lstms.LSTM_0                           -     [79484, 512]  6.30784M   
10_classification.Linear_0      [512, 256]  [64, 1604, 256]  131.328k   
11_classification.Dropout_1              -  [64, 16

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedding.Conv1d_0,"[15, 256, 1]","[64, 256, 1604]",4096.0,6159360.0
1_embedding.BatchNorm1d_1,[256],"[64, 256, 1604]",512.0,256.0
2_embedding.GELU_2,-,"[64, 256, 1604]",NaN,NaN
3_embedding.Conv1d_3,"[256, 512, 1]","[64, 512, 1604]",131584.0,210239488.0
4_embedding.BatchNorm1d_4,[512],"[64, 512, 1604]",1024.0,512.0
5_embedding.GELU_5,-,"[64, 512, 1604]",NaN,NaN
6_embedding.Conv1d_6,"[512, 512, 1]","[64, 512, 1604]",262656.0,420478976.0
7_embedding.BatchNorm1d_7,[512],"[64, 512, 1604]",1024.0,512.0
8_embedding.GELU_8,-,"[64, 512, 1604]",NaN,NaN


# Training Config

In [ ]:
train_config = {
    "beam_width" : 4,
    "lr" : 1e-3,
    "epochs" : 80
    } # Feel free to add more items here

In [ ]:
#TODO


criterion = nn.CTCLoss()# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), train_config['lr']) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(LABELS, beam_width=train_config['beam_width'], log_probs_input=True)#TODO 

# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 200)#TODO
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.7, patience=2,)

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

### Levenshtein

In [ ]:
# Use debug = True to see debug outputs
def calculate_levenshtein(h, y, lh, ly, decoder, labels, debug = False):

    if debug:
        pass
        # print(f"\n----- IN LEVENSHTEIN -----\n")
        # Add any other debug statements as you may need
        # you may want to use debug in several places in this function
        
    # TODO: look at docs for CTC.decoder and find out what is returned here
    results, scores, timesteps, out_seq_len = decoder.decode(h, seq_lens = lh)

    batch_size = y.shape[0] # TODO
    distance = 0 # Initialize the distance to be 0 initially

    for i in range(batch_size):
        # TODO: Loop through each element in the batch
        y_slice = y[i,:ly[i]]
        y_phon = ''.join(LABELS[i] for i in y_slice)
        y_slice_pred = results[i,0,:out_seq_len[i,0]]
        y_phon_pred = ''.join(LABELS[i] for i in y_slice_pred)
        distance += Levenshtein.distance(y_phon, y_phon_pred)

    distance /= batch_size # TODO: Uncomment this, but think about why we are doing this

    return distance

# Training

## wandb

You will need to fetch your api key from wandb.ai

In [ ]:
!pip install wandb -q

     |████████████████████████████████| 1.9 MB 8.3 MB/s 
     |████████████████████████████████| 168 kB 77.9 MB/s 
     |████████████████████████████████| 182 kB 86.1 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 166 kB 90.2 MB/s 
     |████████████████████████████████| 166 kB 89.4 MB/s 
     |████████████████████████████████| 162 kB 90.8 MB/s 
     |████████████████████████████████| 162 kB 64.4 MB/s 
     |████████████████████████████████| 158 kB 97.5 MB/s 
     |████████████████████████████████| 157 kB 92.6 MB/s 
     |████████████████████████████████| 157 kB 87.4 MB/s 
     |████████████████████████████████| 157 kB 95.9 MB/s 
     |████████████████████████████████| 157 kB 90.9 MB/s 
     |████████████████████████████████| 157 kB 98.6 MB/s 
     |████████████████████████████████| 157 kB 91.7 MB/s 
     |████████████████████████████████| 157 kB 86.0 MB/s 
     |████████████████████████████████| 156 kB 96.0 MB/s 


In [ ]:
import wandb
wandb.login(key="0f42a1910060e4a99487f7b1241e7d8a3d45182b")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name = "hw3-med", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account 
    config = train_config ### Wandb Config for your run
)

wandb: Currently logged in as: ram-kasi. Use `wandb login --relogin` to force relogin


# Train

### Eval function
Writing a function to do one round of evaluations will help make your code more modular, you can, however, choose to skip this if you'd like it.

In [ ]:
torch.cuda.empty_cache()
def evaluate(data_loader, model):
    
    model.eval()
    dist = 0
    loss = 0
    batch_bar = tqdm(total=len(data_loader), dynamic_ncols=True, leave=False, position=0, desc='Val')

    # TODO Fill this function out, if you're using it.
    for i, data in enumerate(tqdm(data_loader)):
        m_pad, t_pad, m_len, t_len = data
        m_pad = m_pad.to(device)

        with torch.inference_mode():
            out, out_lengths = model(m_pad, m_len)
            out_permuted = torch.permute(out, (1, 0, 2))
            c_loss = criterion(out_permuted, t_pad, out_lengths, t_len)

        dist += calculate_levenshtein(out, t_pad, out_lengths, t_len, decoder, LABELS, debug = False)
        loss += c_loss.item()

        batch_bar.set_postfix(
              loss = f"{loss/ (i+1):.4f}",
              dist = f"{dist/ (i+1):.4f}"
          )

        batch_bar.update()

        del m_pad, t_pad, m_len, t_len
        del out, out_lengths
        torch.cuda.empty_cache()
  
    batch_bar.close()
    loss /= len(data_loader)
    dist /= len(data_loader)
    
    return loss, dist

### Training Setup

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = train_config['epochs']
best_val_dist = 22 # if you're restarting from some checkpoint, use what you saw there.
dist_freq = 1

Again, writing a train step might help you code be more modular. You may choose to skip this and write the whole thing out in the training loop below if you so wish.

In [ ]:
def train_step(train_loader, model, optimizer, criterion, scheduler, scaler):
    
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    train_loss = 0

    for i, data in enumerate(train_loader):

        # TODO: Fill this with the help of your sanity check
        optimizer.zero_grad()
        m_pad, t_pad, m_len, t_len = data
        m_pad = m_pad.to(device)


        # HINT: Are you using mixed precision? 
        with torch.cuda.amp.autocast():
            out, out_lengths = model(m_pad, m_len)
            out_permuted = torch.permute(out, (1, 0, 2))
            c_loss = criterion(out_permuted, t_pad, out_lengths, t_len)

        train_loss += c_loss.item()

        scaler.scale(c_loss).backward()
        scaler.step(optimizer)
        scaler.update() 

        batch_bar.set_postfix(
            loss = f"{train_loss/ (i+1):.4f}",
            lr = f"{optimizer.param_groups[0]['lr']}"
        )

        batch_bar.update()

        del m_pad, t_pad, m_len, t_len
        del out, out_lengths
        torch.cuda.empty_cache()
    
    batch_bar.close()
    train_loss /= len(train_loader) # TODO

    return train_loss # And anything else you may wish to get out of this function

### Train Loop

In [ ]:
model = Network().to(device)
#model.load_state_dict(torch.load('./hw3_checkpoint.pth')['model_state_dict']) 

In [ ]:
torch.cuda.empty_cache()
gc.collect()
val_dist = best_val_dist

#TODO: Please complete the training loop

for epoch in range(train_config["epochs"]):

    curr_lr = float(optimizer.param_groups[0]['lr'])
    # one training step
    train_loss = train_step(train_loader, model, optimizer, criterion, scheduler, scaler)

    print("\nEpoch {}/{}: \n Train Loss {:.04f}\t Learning Rate {:.04f}".format(
        epoch + 1,
        train_config['epochs'],
        train_loss,
        curr_lr))
    
    # one validation step (if you want)
    val_loss, val_dist = evaluate(val_loader, model)

    print("Val Loss {:.04f}\t Val Dist {:.04f}".format(val_loss, val_dist))

    # HINT: Calculating levenshtein distance takes a long time. Do you need to do it every epoch?
    # Does the training step even need it? 

    # Where you have your scheduler.step depends on the scheduler you use.
    scheduler.step(val_loss)
    
    # Use the below code to save models
    if val_dist < best_val_dist:
      
      print("Saving model")
      torch.save({'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'val_dist': val_dist, 
                  'epoch': epoch}, './hw3_low_checkpoint.pth')
      best_val_dist = val_dist
      wandb.save('checkpoint.pth')
    

    # You may want to log some hyperparameters and results on wandb
    wandb.log({"train_loss":train_loss,'validation_loss': val_loss, 'validation_dist': val_dist, "learning_Rate": curr_lr})

run.finish()

Train:   0%|          | 0/1626 [00:00<?, ?it/s]


Epoch 1/80: 
 Train Loss 0.0957	 Learning Rate 0.0001


Val:   0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

Val Loss 0.2504	 Val Dist 4.1222
Saving model


...


Epoch 5/80: 
 Train Loss 0.0937	 Learning Rate 0.0000


Val:   0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

Val Loss 0.2526	 Val Dist 4.1618


Train:   0%|          | 0/1626 [00:00<?, ?it/s]

# Generate Predictions and Submit to Kaggle

In [ ]:
torch.save(model,'./model_checkpoint_2.pth')

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

decoder_test = CTCBeamDecoder(LABELS, beam_width=train_config['beam_width'], log_probs_input=True)

def make_output(h, lh, decoder, LABELS):
  
    results, scores, timesteps, out_seq_len = decoder.decode(h, lh) #TODO: What parameters would the decode function take in?
    batch_size = h.shape[0]

    dist = 0
    preds = []
    for i in range(batch_size): # Loop through each element in the batch

        h_sliced = results[i,0,:out_seq_len[i,0]] #TODO: Obtain the beam results
        h_string = ''.join(LABELS[j] for j in h_sliced) #TODO: Convert the beam results to phonemes
        preds.append(h_string)
    
    return preds

In [ ]:
#TODO:
# Write a function (predict) to generate predictions and submit the file to Kaggle
def predict(test_loader, model, decoder, LABELS):
    model.eval()
    batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, position=0, leave=False, desc='Test')
    predictions = []

    for i, data in enumerate(tqdm(test_loader)):
        m_pad, m_len = data
        m_pad = m_pad.to(device)

        with torch.inference_mode():
            out, out_lengths = model(m_pad, m_len)
            preds = make_output(out, out_lengths, decoder, LABELS)

        predictions.extend(preds)

        batch_bar.update()

        del m_pad, m_len
        del preds
        torch.cuda.empty_cache()
  
    batch_bar.close()
    return predictions

In [ ]:
torch.cuda.empty_cache()
predictions = predict(test_loader, model, decoder, LABELS)
import pandas as pd

df = pd.read_csv('/content/hw3p2/test-clean/transcript/random_submission.csv')
df.label = predictions

df.to_csv('submission.csv', index = False)
!kaggle competitions submit -c 11-785-f22-hw3p2 -f submission.csv -m "I made it!"

Test:   0%|          | 0/41 [00:00<?, ?it/s]

  0%|          | 0/41 [00:00<?, ?it/s]

100% 210k/210k [00:03<00:00, 60.9kB/s]
Successfully submitted to Automatic Speech Recognition (ASR)